# TS Prophet
## Data split in training and test. The last 365 days are the test data.

In [1]:
from audioop import cross
import itertools
from matplotlib import units
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import add_changepoints_to_plot
from prophet.plot import plot_cross_validation_metric
from prophet.plot import plot_plotly, plot_components_plotly

plt.style.use('fivethirtyeight')

from sklearn.metrics import mean_absolute_error,mean_squared_error

from green_city.utils import metrics_dict, datetime2index, index2datetime

# Libraries to reduce Prophet verbose output
import warnings
import logging
import os

%matplotlib inline

In [2]:
## MLFLOW ##
import mlflow
from green_city.mlflow_config import get_mlflow_config

flow_conf = get_mlflow_config()
tracking_uri = flow_conf["TRACKING_URI"]
mlflow.set_tracking_uri(flow_conf["TRACKING_URI"])
mlflow.set_experiment(flow_conf["EXPERIMENT_NAME"]);

In [3]:
def is_winter_season(ds):
    """Classifies the dates into Winter or Summer

    Args:
        ds (datetime series): A Pandas datetime series
    """
    date = pd.to_datetime(ds)
    return(date.month < 4 or date.month > 9)


In [4]:
## DB CONNECTION ##
from sqlalchemy import create_engine
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv("../.db_credentials"))

db_connection_credentials = {
    "database": config('POSTGRES_DB'),
    "user": config('POSTGRES_USER'),
    "password": config('POSTGRES_PASSWORD'),
    "host": config('POSTGRES_HOST'),
    "port": config('POSTGRES_PORT'),
}
DB_STRING = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_connection_credentials)
db = create_engine(DB_STRING)

### Load data and feature engineering

In [5]:
building = 5
# Load data
df_building = pd.read_csv(f"../data/preprocessed/Building_{building}.csv").astype({'datetime': 'datetime64'})#.set_index('datetime')

In [ ]:
# df_building['net_load_kWh'].plot()


In [ ]:
sns.histplot(df_building['net_load_kWh'], bins=30)

In [6]:
df_building = df_building.rename(columns={'datetime':'ds',
                                    'net_load_kW':'y'})
# df.head(5)

In [7]:
df_building['winter'] = df_building['ds'].apply(is_winter_season)
df_building['summer'] = ~df_building['ds'].apply(is_winter_season)

In [8]:
df_holiday = df_building[['ds','holiday']].query('holiday')
df_holiday['holiday'] = 'Holiday'
# df_holiday

## In the development stage, it was explored to add the workday as part of the  ##
## seasonality. It did not had an effect. Including the holiday did.            ##
# df_workday = df[['ds','workday']].query('workday')
# df_workday['holiday'] = 'Workday'
# df_workday = df_workday.drop('workday', axis=1)

# df_holidays = pd.concat([df_holiday, df_workday], ignore_index=True)

In [9]:
pred_indices = [32135, 33311, 26478, 33357, 30387, 30794, 31800, 28783]
# pred_indices = [32135]

## Prophet model. Full data

In [ ]:
my_model_full = Prophet(interval_width=0.95, seasonality_mode='additive')
my_model_full.fit(df_building)

In [ ]:
future_dates = my_model_full.make_future_dataframe(periods = 365)
forecast = my_model_full.predict(future_dates)


In [ ]:
fig = my_model_full.plot(forecast, uncertainty=True, plot_cap=False)
ax = fig.gca()
ax.set_xlim(pd.to_datetime(['2008-01-02', '2009-01-01']))

In [ ]:
sns.pointplot(x = 'ds', y='y', data=df_building[:24*365])
# sns.lineplot(x = 'ds', y='yhat', data=forecast[:24*365])

## Prophet simple model. Three years training

### Split data in training and test data

In [ ]:
df_simple_model_train = df_building[:365*3*24]
df_simple_model_test = df_building[365*3*24:]

### Simple Prophed model. Specify 95% of uncertainty and additive model. (Default 80% and additive model)

In [ ]:
my_model_simple = Prophet(interval_width = 0.95, seasonality_mode="additive")

In [ ]:
my_model_simple.fit(df_simple_model_train)

In [ ]:
for index in pred_indices:
    with mlflow.start_run(run_name='Prophet Simple Model') as r:
        # print(r.info.run_id)
        ### Predict ##
        # my_model_simple.fit(df.loc[:index])
        df_future = df_building.copy().loc[index+1:index+24]
        df_forecast = my_model_simple.predict(df_future)
        df_forecast.index = df_future.index
        df_forecast.index.name = 'id'
        df_forecast['error'] = df_future.y - df_forecast.yhat
        df_forecast = df_forecast[['yhat', 'error']]
        
        ## Evaluate ##
        metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
        print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
        mlflow.log_metrics(metrics)

        ## Parameters log ##
        params = {
            'building_nr': building,
            'datetime':index2datetime(index)
        }
        mlflow.log_params(params)

        forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
        # print(forecasts.head())
        # forecasts.to_sql("forecast", con=db, if_exists="append")

## Optimize modeling

In [10]:
def my_prophet_mse(df, all_params):
    """Determine the best hyper parameters for a Prophet model.
    It uses a base Prophet model. 

    Args:
        df (dataframe): A pandas dataframe with the required Prophet structure
        all_params (list of dictionaries): A list of dictionaries with the parameters definition. 

    Returns:
        best_params: a list with the best parameters
        tuning_results: A dataframe with the mse and rmse evaluation of the parameters combinations
    """
    warnings.simplefilter("ignore", DeprecationWarning)
    warnings.simplefilter("ignore", FutureWarning)
    logging.getLogger('prophet').setLevel(logging.ERROR) #Notice that i had modified the name from 'fbprophet' to just 'prophet'
    mses = []
    rmses = []
    for params in all_params:
        m = Prophet(**params)
        m.fit(df)
        df_cv = cross_validation(m, horizon='180 days', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        mses.append(df_p['mse'].values[0])
        rmses.append(df_p['rmse'].values[0])
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mse'] = mses
    tuning_results['rmse'] = rmses
    print(tuning_results)
    best_params=all_params[np.argmin(mses)]
    print(best_params)
    return best_params, tuning_results


In [ ]:
# def my_prophet_rmse(df, all_params):
#     # warnings.simplefilter("ignore", DeprecationWarning)
#     # warnings.simplefilter("ignore", FutureWarning)
#     # logging.getLogger('prophet').setLevel(logging.ERROR) #Notice that i had modified the name from 'fbprophet' to just 'prophet'
#     rmses = []
#     for params in all_params:
#         m = Prophet(**params)
#         m.fit(df)
#         df_cv = cross_validation(m, horizon='180 days', parallel='processes')
#         df_p = performance_metrics(df_cv, rolling_window=1)
#         rmses.append(df_p['rmse'].values[0])
#     tuning_results = pd.DataFrame(all_params)
#     tuning_results['rmse'] = rmses
#     print(tuning_results)
#     best_params=all_params[np.argmin(rmses)]
#     print(best_params)
#     return best_params, tuning_results


In [11]:
def my_best_params(df, best_params):
    """ Fits the best hyper parameters to the Prophet model

    Args:
        df (dataframe): A pandas dataframe to be fitted
        best_params (list): A list with the best parameters to fit. It must be a list of a dictionary.

    Returns:
        my_model: A Prophet fitted model.
    """
    my_model = Prophet(**best_params)
    my_model.fit(df)
    return my_model

In [15]:
def my_pred_mse(df,model_name, parameters, pred_index):
    """Forecast a specified interval into a Prophet model. 

    Args:
        df (dataframe)    : A pandas dataframe with the form required by Prophet. Must include the training and testing data.
        model_name (str)  : A string with a unique description of the model to evaluate.
        parameters (list) : A list of dictionaries with the parameters definition. 
        pred_index (list) : A list with the indexes of the intervals to predict. ("Test data"). These indexes must exist in the df variable.

    Returns:
        forecasts (dataframe)           : A dataframe with the forecasted values from the interval of interest. 
        ls_best_params (list)           : A list with the best hyperpameters and its evaluation metrics; mse and rsme.
        my_model(fitted Prophet model)  : A fitted prophet model. The model utilized to forecast the data. 
        ls_publish                      : A list with the information to put in mlflow and the SQL server. Includes the forecasts dataframe,
                                          the msa, mse and r2_score.
    """

    ls_best_params = []
    ls_publish = []
    for index in pred_index:
        df_train=df.loc[:index]
        df_future = df.loc[index+1:index+24]
        with mlflow.start_run(run_name=model_name) as r:
            ### Predict ##
            best_params, tuning_results = my_prophet_mse(df_train, all_params=parameters)
            ls_best_params.append([{'index':index}, best_params, tuning_results])
            my_model = my_best_params(df_train, best_params = best_params)
            df_forecast = my_model.predict(df_future)
            df_forecast.index = df_future.index
            df_forecast.index.name = 'id'
            df_forecast['error'] = df_future.y - df_forecast.yhat
            df_forecast = df_forecast[['yhat', 'error']]
            
            ## Evaluate ##
            metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
            print(index)
            print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
            print(ls_best_params)
            mlflow.log_metrics(metrics)

            ## Parameters log ##
            params = {
                'building_nr': building,
                'datetime':index2datetime(index)
            }
            mlflow.log_params(params)

            forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
            # forecasts.to_sql("forecast", con=db, if_exists="append")
            ls_publish.append([{'index':index,'metrics':metrics, 'params':params, 'forecast':forecasts}])
    return forecasts, ls_best_params, my_model, ls_publish

In [ ]:
# def my_pred_rsme(df,model_name, parameters, pred_index):
#     ls_best_params = []
#     for index in pred_index:
#         with mlflow.start_run(run_name=model_name) as r:
#             ### Predict ##
#             best_params, tuning_results = my_prophet_rmse(df=df.loc[:index], all_params=parameters)
#             ls_best_params.append([{'index':index}, best_params, tuning_results])
#             my_model = my_best_params(df=df.loc[:index], best_params = best_params)
#             df_future = df.loc[index+1:index+24]
#             df_forecast = my_model.predict(df_future)
#             df_forecast.index = df_future.index
#             df_forecast.index.name = 'id'
#             df_forecast['error'] = df_future.y - df_forecast.yhat
#             df_forecast = df_forecast[['yhat', 'error']]
            
#             ## Evaluate ##
#             metrics = metrics_dict(df_future.y, df_forecast.yhat, ["mae", "mse", "r2_score"])
#             print(index)
#             print("mae: {mae}, mse: {mse}, r2: {r2_score}".format(**metrics))
#             print(ls_best_params)
#             mlflow.log_metrics(metrics)

#             ## Parameters log ##
#             params = {
#                 'building_nr': building,
#                 'datetime':index2datetime(index)
#             }
#             mlflow.log_params(params)

#             forecasts = df_forecast[['yhat']].assign(run_id = r.info.run_id).rename(columns={"yhat": "prediction"})
#             # forecasts.to_sql("forecast", con=db, if_exists="append")
#     return forecasts, ls_best_params

In [ ]:
# # Use RMSE for tunning 
# param_grid_simple = {
#     'interval_width': [0.95],
#     'seasonality_mode':['additive'],
#     'changepoint_prior_scale':[0.05],
#     'seasonality_prior_scale':[10.0],
#     'holidays_prior_scale':[0.01]
# }
# all_params = [dict(zip(param_grid_simple.keys(), v)) for v in itertools.product(*param_grid_simple.values())]

# pred_rmse, ls_best_rmse = my_pred_rsme(df=df_building,
#                 model_name='Prophet Simple Model w/Params - Tunning RMSE', 
#                 parameters=all_params,
#                 pred_index=pred_indices)

In [13]:
# Use MSE for tunning 
param_grid_simple = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.05],
    'seasonality_prior_scale':[10.0],
    'holidays_prior_scale':[0.01],
    'changepoint_range':[0.8]#,0.9]
}
all_params = [dict(zip(param_grid_simple.keys(), v)) for v in itertools.product(*param_grid_simple.values())]

pred_mse_simpleMod_Opt, ls_best_mse_simpleMod_Opt, m_mse_simpleMod_Opt, ls_publish_mse_simpleMod_Opt = my_pred_mse(df = df_building,
                model_name='Prophet Simple Model w/Params - Tunning MSE', 
                parameters=all_params,
                pred_index=pred_indices)

Initial log joint probability = -566.475
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       56653.8    0.00416767       1175.87           1           1      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       56676.6    0.00040963       735.747      0.3855     0.03855      235   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299         56682   0.000290279       449.935      0.1605      0.6624      349   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       56687.2     0.0153445       618.011           1           1      463   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       56689.3   0.000194497       194.598      0.7101      0.7101      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [14]:
ls_best_mse_simpleMod_Opt

[[{'index': 32135},
  {'interval_width': 0.95,
   'seasonality_mode': 'additive',
   'changepoint_prior_scale': 0.05,
   'seasonality_prior_scale': 10.0,
   'holidays_prior_scale': 0.01,
   'changepoint_range': 0.8},
     interval_width seasonality_mode  changepoint_prior_scale  \
  0            0.95         additive                     0.05   
  
     seasonality_prior_scale  holidays_prior_scale  changepoint_range  \
  0                     10.0                  0.01                0.8   
  
           mse      rmse  
  0  73.974423  8.600839  ]]

### Parameters that can be tuned

**changepoint_prior_scale**: This is probably the most impactful parameter. It determines the flexibility of the trend, and in particular how much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.

**seasonality_prior_scale**: This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

**holidays_prior_scale**: This controls flexibility to fit holiday effects. Similar to seasonality_prior_scale, it defaults to 10.0 which applies basically no regularization, since we usually have multiple observations of holidays and can do a good job of estimating their effects. This could also be tuned on a range of [0.01, 10] as with seasonality_prior_scale.

**seasonality_mode**: Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series (see the documentation here on multiplicative seasonality), but when that isn’t possible, it could be tuned.

In [ ]:
# # Parameter optimization. Tunning CV with RMSE 
# param_grid = {
#     'interval_width': [0.95],
#     'seasonality_mode':['additive'],
#     'changepoint_prior_scale':[0.0001, 0.01, 0.05, 0.1, 0.5], #Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.
#     'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0], #This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.
#     'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
#     'changepoint_range':[0.8,0.9]
# }
# all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

# pred_rmse, ls_best_rmse = my_pred_rsme(df = df_building,
#         model_name='Prophet Simple Model Optimized Parameters - Tunning with RMSE', 
#         parameters=all_params,
#         pred_index=pred_indices)

In [ ]:
# Parameter optimization. Tunning CV with MSE 
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001, 0.01, 0.05, 0.1, 0.5], #Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0], #This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.
    'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'changepoint_range':[0.8,0.9]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_mse_simpleMod_Opt, ls_best_mse_simpleMod_Opt, m_mse_simpleMod_Opt, ls_publish_mse_simpleMod_Opt = my_pred_mse(df = df_building,
        model_name='Prophet Simple Model Optimized Parameters - Tunning with MSE', 
        parameters=all_params,
        pred_index=pred_indices)

### Comparison between RMSE and MSE for CV assessment in Hyperparmeter selection

### Add holidays

In [ ]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'holidays':[df_holiday],
    'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'changepoint_range':[0.8,0.9]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_mse_WkMod_Opt_holidays, ls_best_mse_WkMod_Opt_holidays, m_best_mse_WkMod_Opt_holidays, ls_publish_best_mse_WkMod_Opt_holidays = my_pred_mse(df = df_building,
        model_name='Prophet Holidays-Optimized-Parameters - Tunning with MSE', 
        parameters=all_params,
        pred_index=pred_indices)

### Assume different weekly sesonalities. Summer vs Winter

In [ ]:
def my_prophet_mse(df, all_params):
    """Determine the best hyper parameter models.
    It specifies a weekly seasonality based in two different seasons; Winter and Summer. Prophet model. 

    Args:
        df (dataframe): A pandas dataframe with the required Prophet structure
        all_params (list of dictionaries): A list of dictionaries with the parameters definition. 

    Returns:
        best_params: a list with the best parameters
        tuning_results: A dataframe with the mse and rmse evaluation of the parameters combinations
    """
    warnings.simplefilter("ignore", DeprecationWarning)
    warnings.simplefilter("ignore", FutureWarning)
    logging.getLogger('prophet').setLevel(logging.ERROR) #Notice that i had modified the name from 'fbprophet' to just 'prophet'
    mses = []
    rmses = []
    for params in all_params:
        m = Prophet(**params,
        weekly_seasonality=False
        )
        m.add_seasonality(name='weekly_on_winter', period = 7, fourier_order= 10, condition_name='winter')
        m.add_seasonality(name='weekly_on_summer', period = 7, fourier_order= 10, condition_name='summer')
        m.fit(df)
        df_cv = cross_validation(m, horizon='180 days', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        mses.append(df_p['mse'].values[0])
        rmses.append(df_p['rmse'].values[0])
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mse'] = mses
    tuning_results['rmse'] = rmses
    print(tuning_results)
    best_params=all_params[np.argmin(rmses)]
    print(best_params)
    return best_params, tuning_results

In [ ]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'holidays':[df_holiday],
    'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'changepoint_range':[0.8,0.9]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_mse_WkSeasMod_Opt, ls_best_mse_WkSeasMod_Opt, m_best_mse_WkSeasMod_Opt, ls_publish_best_mse_WkSeasMod_Opt = my_pred_mse(df=df_building,
        model_name='Prophet Holidays-Winter/Summer-Optimized-Parameters - Tunning with MSE', 
        parameters=all_params,
        pred_index=pred_indices)

### Add outdoor temperature and direct solar energy as additional regressor. Include 24h forecast for each feature too. 
### The temperature and solar energy data is in the original dataset.

In [16]:
def my_prophet_mse(df, all_params):
    """Determine the best hyper parameter models.
    It specifies a weekly seasonality based in two different seasons; Winter and Summer. 
    It specifies additional regressors to the Prophet model. 

    Args:
        df (dataframe): A pandas dataframe with the required Prophet structure
        all_params (list of dictionaries): A list of dictionaries with the parameters definition. 

    Returns:
        best_params: a list with the best parameters
        tuning_results: A dataframe with the mse and rmse evaluation of the parameters combinations
    """
    warnings.simplefilter("ignore", DeprecationWarning)
    warnings.simplefilter("ignore", FutureWarning)
    logging.getLogger('prophet').setLevel(logging.ERROR) #Notice that i had modified the name from 'fbprophet' to just 'prophet'
    mses = []
    rmses = []
    for params in all_params:
        m = Prophet(**params,
        weekly_seasonality=False
        )
        m.add_seasonality(name='weekly_on_winter', period = 7, fourier_order= 10, condition_name='winter')
        m.add_seasonality(name='weekly_on_summer', period = 7, fourier_order= 10, condition_name='summer')
        m.add_regressor(name='outdoor_temp')
        m.add_regressor(name='pred_24h_outdoor_temp')
        m.add_regressor(name='direct_solar_W_m2')
        m.add_regressor(name='pred_24h_direct_solar_W_m2')
        m.fit(df)
        df_cv = cross_validation(m, horizon='180 days', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        mses.append(df_p['mse'].values[0])
        rmses.append(df_p['rmse'].values[0])
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mse'] = mses
    tuning_results['rmse'] = rmses
    print(tuning_results)
    best_params=all_params[np.argmin(rmses)]
    print(best_params)
    return best_params, tuning_results

In [17]:
param_grid = {
    'interval_width': [0.95],
    'seasonality_mode':['additive'],
    'changepoint_prior_scale':[0.0001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'holidays':[df_holiday],
    'holidays_prior_scale':[0.01, 0.1, 1.0, 10.0],
    'changepoint_range':[0.8,0.9]
}
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

pred_mse_best_mse_RegMod_Opt, ls_best_mse_best_mse_RegMod_Opt, m_mse_best_mse_RegMod_Opt, ls_publish_mse_best_mse_RegMod_Opt = my_pred_mse(df=df_building,
        model_name='Prophet Holidays-Winter/Summer-Regressors-Optimized-Parameters - Tunning with MSE', 
        parameters=all_params,
        pred_index=pred_indices)

Initial log joint probability = -566.475
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       62253.3   7.66563e-06       51099.3       0.874       0.874      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     121       62257.8   5.04082e-05       50316.9   1.009e-09       0.001      210  LS failed, Hessian reset 
     159       62260.6   2.21114e-06       50716.3    4.32e-11       0.001      306  LS failed, Hessian reset 
     193       62260.7   8.40992e-09       49595.5       0.795       0.795      348   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -415.366
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial log joint probability = -376.544
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Initial 

In [18]:
pred_mse_best_mse_RegMod_Opt

,prediction,run_id
id,,
32136,29.077647,d505347f8c394f9888c608992604065e
32137,27.065611,d505347f8c394f9888c608992604065e
32138,24.963615,d505347f8c394f9888c608992604065e
32139,24.364733,d505347f8c394f9888c608992604065e
32140,26.739614,d505347f8c394f9888c608992604065e
32141,31.884745,d505347f8c394f9888c608992604065e
32142,37.629810,d505347f8c394f9888c608992604065e
32143,41.190362,d505347f8c394f9888c608992604065e
32144,41.083677,d505347f8c394f9888c608992604065e


In [19]:
ls_best_mse_best_mse_RegMod_Opt

[[{'index': 32135},
  {'interval_width': 0.95,
   'seasonality_mode': 'additive',
   'changepoint_prior_scale': 0.0001,
   'seasonality_prior_scale': 1.0,
   'holidays':                        ds  holiday
   0     2008-01-02 00:00:00  Holiday
   1     2008-01-02 01:00:00  Holiday
   2     2008-01-02 02:00:00  Holiday
   3     2008-01-02 03:00:00  Holiday
   4     2008-01-02 04:00:00  Holiday
   ...                   ...      ...
   34891 2011-12-25 19:00:00  Holiday
   34892 2011-12-25 20:00:00  Holiday
   34893 2011-12-25 21:00:00  Holiday
   34894 2011-12-25 22:00:00  Holiday
   34895 2011-12-25 23:00:00  Holiday
   
   [960 rows x 2 columns],
   'holidays_prior_scale': 0.1,
   'changepoint_range': 0.8},
       interval_width seasonality_mode  changepoint_prior_scale  \
  0              0.95         additive                   0.0001   
  1              0.95         additive                   0.0001   
  2              0.95         additive                   0.0001   
  3             

In [20]:
m_mse_best_mse_RegMod_Opt

In [21]:
ls_publish_mse_best_mse_RegMod_Opt

[[{'index': 32135,
   'metrics': {'mae': 5.7422959925925285,
    'mse': 49.55838075217169,
    'r2_score': 0.3469757490357219},
   'params': {'building_nr': 5,
    'datetime': datetime.datetime(2011, 9, 1, 23, 0)},
   'forecast':        prediction                            run_id
   id                                                 
   32136   29.077647  d505347f8c394f9888c608992604065e
   32137   27.065611  d505347f8c394f9888c608992604065e
   32138   24.963615  d505347f8c394f9888c608992604065e
   32139   24.364733  d505347f8c394f9888c608992604065e
   32140   26.739614  d505347f8c394f9888c608992604065e
   32141   31.884745  d505347f8c394f9888c608992604065e
   32142   37.629810  d505347f8c394f9888c608992604065e
   32143   41.190362  d505347f8c394f9888c608992604065e
   32144   41.083677  d505347f8c394f9888c608992604065e
   32145   38.033842  d505347f8c394f9888c608992604065e
   32146   34.213895  d505347f8c394f9888c608992604065e
   32147   31.615422  d505347f8c394f9888c608992604065e
   